In [68]:
import numpy as np
import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator , TransformerMixin

from sklearn.linear_model import LinearRegression

In [ ]:
class CombinedAttributesAdder(BaseEstimator , TransformerMixin):
    def fit(self , X , y=None):
        
        


In [14]:
x_train = pd.read_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\ScaledTrainSet_data.csv' , index_col  = 'index')
x_train

,year,is_GAC,is_maserati,is_daihatsu,is_porsche,is_ds,is_smart,is_seres,is_cadillac,is_דודג',...,is_leasing,is_company,is_private,next_test_in,annual_licensing_fee,car_length,large,small,mini,medium
index,,,,,,,,,,,,,,,,,,,,,
2902,0.857143,0,0,0,0,0,0,0,0,0,...,0,0,1,0.500000,0.645547,0.562429,0,1,0,0
73,0.857143,0,0,0,0,0,0,0,0,0,...,0,0,1,0.833333,0.295767,0.798861,1,0,0,0
1537,0.809524,0,0,0,0,0,0,0,0,0,...,1,0,0,0.583333,0.188003,0.673624,0,0,0,1
2638,0.761905,0,0,0,0,0,0,0,0,0,...,1,0,0,0.166667,0.188003,0.592030,0,0,0,1
621,0.809524,0,0,0,0,0,0,0,0,0,...,0,0,1,0.416667,0.122825,0.588994,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,0.904762,0,0,0,0,0,0,0,0,0,...,1,0,0,0.416667,0.645547,0.798861,1,0,0,0
1095,0.476190,0,0,0,0,0,0,0,0,0,...,0,0,1,0.000000,0.219683,0.733966,0,0,0,1
1130,0.809524,0,0,0,0,0,0,0,0,0,...,1,0,0,0.666667,0.188003,0.732827,0,0,0,1


# Cross-validation :

In [ ]:
#first we return the demi-model (the average) and try to be better than him.... 

In [63]:
x_target = pd.read_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\TrainSet-target_data.csv' , index_col='index' )
x_target.mean() 
#demi-model : return  111097.128101

price    111097.128101
dtype: float64

In [64]:
# x_target[["price"]] = MinMaxScaler().fit_transform(x_target[["price"]]) 
# x_target = x_target["price"]
x_target

,price
index,
2902,217000
73,169900
1537,94900
2638,76000
621,86500
...,...
1638,192900
1095,83500
1130,78900


In [65]:
#את הדאטה שהקדשנו לטריין אנחנו מפצלים שוב ע"מ לבחון את מודל הרגרסיה הלינארית עם רנדום 1

x_train1 , x_test1 , y_train1 , y_test1= sklearn.model_selection.train_test_split(x_train, x_target , test_size=0.1 , random_state = 1) 

lin_reg =  LinearRegression()
fit1= lin_reg.fit(x_train1, y_train1)
pred = fit1.predict(x_test1)
print (f"\n{pred}")
print (f"RMSA: {np.sqrt(mean_squared_error(y_test1 , pred))}\n")



[[ 26944. ]
 [ 53925.5]
 [162080. ]
 [ 77065.5]
 [103235.5]
 [ 13206.5]
 [113305. ]
 [ 46216.5]
 [108448. ]
 [ 44852.5]
 [456571.5]
 [136619. ]
 [ 26527.5]
 [106046.5]
 [113290.5]
 [216493.5]
 [ 12639.5]
 [ 25575. ]
 [ 20546. ]
 [ 83572.5]
 [ 66883.5]
 [142194. ]
 [ 67027. ]
 [ 67441. ]
 [ 78095.5]
 [128279.5]
 [ 45925.5]
 [ 43819.5]
 [ 49145.5]
 [376673.5]
 [ 56111.5]
 [126338.5]
 [ 46606.5]
 [ 90321.5]
 [ 43141. ]
 [127423.5]
 [ 83752. ]
 [ 72863.5]
 [ 54635.5]
 [ 43409.5]
 [186810.5]
 [ 66929. ]
 [134060. ]
 [ 77476.5]
 [ 68907. ]
 [332200. ]
 [ 84427.5]
 [ 57250. ]
 [ 46815.5]
 [106966.5]
 [348005.5]
 [ 62837.5]
 [ 62690.5]
 [202578. ]
 [ 64875.5]
 [ 69384.5]
 [ 83834. ]
 [272509.5]
 [ 97169.5]
 [ 96108.5]
 [112756.5]
 [ 86497. ]
 [143449.5]
 [139313.5]
 [ 17325.5]
 [ 96748.5]
 [ -5688.5]
 [ 78199. ]
 [ 53826. ]
 [ 18491. ]
 [149157.5]
 [122424.5]
 [133560. ]
 [134510. ]
 [ 61905.5]
 [107392.5]
 [ 45902. ]
 [107236. ]
 [ 46244.5]
 [109167.5]
 [ 77298. ]
 [  6740.5]
 [150907.5]
 [ 

In [67]:
y_test1

,price
index,
1201,30000
2869,97900
2561,99900
258,59900
2858,109900
...,...
1397,179900
661,46000
2445,290000


In [ ]:

#מנסים להבין למה לא הצךלחנו לחזות טוב עם המודל שלנו... עוברים על המכוניות ידנית ורואים באילו מכוניות הטעויות הגדולות קרו
#ננסה לבנות עמודה שהקורלציה שלה עם המחיר תהיה גבוהה
#ננסה לשנות את ה"טראש הולד" של קביעת גודל המכונית ולהכניס מכוניות לקטגוריה "גדול" רק מסף גבוהה יותר

In [66]:
# scores = cross_val_score(lin_reg, x_train, x_target, scoring= 'neg_mean_squared_error' ,cv=3)
# np.sqrt(-scores)

array([1.01107624e+16, 4.78137929e+04, 1.33658746e+15])

In [70]:
x_train1 , x_test1 , y_train1 , y_test1= sklearn.model_selection.train_test_split(x_train, x_target , test_size=0.1 , random_state = 1) 

tree_reg =  DecisionTreeRegressor()
fit1 = tree_reg.fit(x_train1 , y_train1)
pred = fit1.predict(x_test1)
print (f"\n{pred}")
print (f"RMSA: {np.sqrt(mean_squared_error(y_test1 , pred))}\n")



[ 43000. 104900. 168000.  63500. 104900.  29000. 120990.  20900. 105000.
  47900. 435000. 124900.  26000.  85900. 121900. 138000.  32000.  54000.
  29000.  88000.  39900.  74920.  85500.  68412.  67000. 109900.  42540.
  37000. 104900. 435000.  64000.  99900.  42540. 119900.  52900.  96500.
  88000.  85000.  58000.  37400. 134990.  63000. 105000.  47000.  95000.
 219000.  63500.  79000.  46885. 139000. 269000.  62374.  72000. 131000.
  38000.  49500.  94900. 208000. 165900. 130000. 110000.  65000. 155000.
 178000.  25000.  98000.  35000.  89000.  32000.  29000. 135900. 105000.
 155000. 144900.  75900. 108900.  58500. 108900.  62500.  99000.  78900.
  39000. 158900.  79990. 195000.  39000. 175000. 188000.  79900.  82400.
 118000.  90000. 395000.  49000.  73900.  17999.  92000.  24900.  70000.
 122000. 118000.  60000. 120000.  69000.  85500.  50000. 120000. 430000.
  57000. 250000.  52000. 113000. 650000.  16000.  47900.  47900. 124900.
 120990.  91000.  41500. 101000.  91900.  71990.  